In [7]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [8]:
dataset=pd.read_csv("data.csv",error_bad_lines=False)

b'Skipping line 2810: expected 2 fields, saw 5\nSkipping line 4641: expected 2 fields, saw 5\nSkipping line 7171: expected 2 fields, saw 5\nSkipping line 11220: expected 2 fields, saw 5\nSkipping line 13809: expected 2 fields, saw 5\nSkipping line 14132: expected 2 fields, saw 5\nSkipping line 14293: expected 2 fields, saw 5\nSkipping line 14865: expected 2 fields, saw 5\nSkipping line 17419: expected 2 fields, saw 5\nSkipping line 22801: expected 2 fields, saw 5\nSkipping line 25001: expected 2 fields, saw 5\nSkipping line 26603: expected 2 fields, saw 5\nSkipping line 26742: expected 2 fields, saw 5\nSkipping line 29702: expected 2 fields, saw 5\nSkipping line 32767: expected 2 fields, saw 5\nSkipping line 32878: expected 2 fields, saw 5\nSkipping line 35643: expected 2 fields, saw 5\nSkipping line 36550: expected 2 fields, saw 5\nSkipping line 38732: expected 2 fields, saw 5\nSkipping line 40567: expected 2 fields, saw 5\nSkipping line 40576: expected 2 fields, saw 5\nSkipping line 

In [9]:
dataset["strength"].unique()

array([1, 2, 0], dtype=int64)

In [10]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669640 entries, 0 to 669639
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   password  669639 non-null  object
 1   strength  669640 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 10.2+ MB


In [11]:
dataset.isnull().sum()

password    1
strength    0
dtype: int64

In [12]:
dataset[dataset["password"].isnull()]

,password,strength
367579,NaN,0


In [13]:
dataset.dropna(inplace=True)

In [14]:
password_=np.array(dataset)
password_

array([['kzde5577', 1],
       ['kino3434', 1],
       ['visi7k1yr', 1],
       ...,
       ['184520socram', 1],
       ['marken22a', 1],
       ['fxx4pw4g', 1]], dtype=object)

In [15]:
import random
random.shuffle(password_)

In [16]:
password_

array([['kzde5577', 1],
       ['kzde5577', 1],
       ['kino3434', 1],
       ...,
       ['sonia83', 0],
       ['6776828lara', 1],
       ['53787vasco', 1]], dtype=object)

In [17]:
X=[passwords[0] for passwords in password_]
y=[passwords[1] for passwords in password_]

In [18]:
def make_chars(input):
    return [chars for chars in input]

In [19]:
make_chars("password")

['p', 'a', 's', 's', 'w', 'o', 'r', 'd']

In [20]:
vectorizer=TfidfVectorizer(tokenizer=make_chars)

In [21]:
X_=vectorizer.fit_transform(X)

In [22]:
X_.shape

(669639, 129)

In [23]:
first_=X_[0].T.todense()

In [24]:
vec=pd.DataFrame(first_,index=vectorizer.get_feature_names(),columns=['tfidf'])

In [25]:
vec.sort_values(by=['tfidf'],ascending=False)

,tfidf
7,0.591375
5,0.567177
z,0.336091
k,0.291897
d,0.285367
...,...
>,0.000000
=,0.000000
<,0.000000
;,0.000000


In [26]:
x_train,x_test,y_train,y_test=train_test_split(X_,y,test_size=0.2,random_state=42)

In [27]:
x_train.shape,x_test.shape

((535711, 129), (133928, 129))

In [28]:
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.naive_bayes import MultinomialNB

In [29]:
classifier=[]
classifier.append(LogisticRegression(multi_class='ovr',n_jobs=-1))
classifier.append(LogisticRegression(multi_class='multinomial',solver='newton-cg',n_jobs=-1))
classifier.append(xgb.XGBClassifier(n_jobs=-1))
classifier.append(MultinomialNB())

In [30]:
result=[]
for model in classifier:
    a=model.fit(x_train,y_train)
    result.append(a.score(x_test,y_test))

[22:31:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [31]:
result1=pd.DataFrame({'score':result,
                      'algorithms':['logistic_regr_ovr',
                                    'logistic_regr_mutinomial',
                                    'xgboost','naive bayes']})
result1

,score,algorithms
0,0.811100,logistic_regr_ovr
1,0.817887,logistic_regr_mutinomial
2,0.986231,xgboost
3,0.741368,naive bayes


In [32]:
xgb_classifier=xgb.XGBClassifier(n_jobs=-1)
xgb_classifier.fit(x_train,y_train)
pred=xgb_classifier.predict(x_test)

[22:38:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [33]:
import dill
model_file=open("xgb_classifier.pkl","wb")
dill.dump(xgb_classifier,model_file)
model_file.close()
dill.dump(vectorizer, open("vectorizer.pkl", "wb"))

In [34]:
password="password$TeSt&123"
password=vectorizer.transform([password])
xgb_classifier.predict(password)

array([2])